In [3]:
! pip install torchmetrics

   ---------------------------------------- 0.0/926.4 kB ? eta -:--:--
   --------------------------------------- 926.4/926.4 kB 14.1 MB/s eta 0:00:00


In [1]:
import os
import cv2
import yaml
import torch
import numpy as np
import pandas as pd
from torch import nn
import mediapipe as mp
from torch import optim
from datetime import datetime
from torchmetrics import Accuracy
from torch.utils.data import Dataset

In [2]:
def label_dict_from_config_file(relative_path):
    with open(relative_path,"r") as f:
       label_tag = yaml.full_load(f)["gestures"]
    return label_tag

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        list_label = label_dict_from_config_file("../data/hand_gesture.yaml")

        '''Hoàn thành đoạn code để xây dựng một model gồm có 4 hidden layer,
            lần lượng input và output là (63, 128), (128, 128), (128, 128), (128, 128).
            Layer đầu tiên được theo sau bổi một Relu và Batchnorm1d.
            Layer thứ 2, 3, và 4 được theo sau bỏi Relu và Dropout với rate lần lượt là 0.4, 0.4, 0.6.
            Output layer có nhịêm vụ phân loại với input là 128 và output là số lượng class cử chỉ
        '''
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(63, 128), 
            nn.ReLU(),              
            nn.BatchNorm1d(128),    
            nn.Linear(128, 128),   
            nn.ReLU(),         
            nn.Dropout(0.4),      
            nn.Linear(128, 128),     
            nn.ReLU(),               
            nn.Dropout(0.4),         
            nn.Linear(128, 128),     
            nn.ReLU(),               
            nn.Dropout(0.6),        
            nn.Linear(128, list_label)
        )
    def forward(self, x):

        ''' Hoàn thành code để thực hiện forward dự đoán cử chỉ với input x.
        Thực hiệnt flatten x
        Pass x vừa flatten vào linear_relu_stack
        Return  logits (outputs từ layer cuối cùng)
        '''

    def predict(self,x,threshold=0.8):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits)
        chosen_ind = torch.argmax(softmax_prob,dim=1)
        return torch.where(softmax_prob[0,chosen_ind]>threshold,chosen_ind,-1)

    def predict_with_known_class(self,x):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits)
        return torch.argmax(softmax_prob,dim=1)

    def score(self,logits):
        return -torch.amax(logits,dim=1)